In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from matplotlib import pyplot
import tensorflow as tf

# Tensorflow estimator Model.

In [70]:
def my_model_fn(features, labels, mode, params):
    
    print (features)
    # Creating the model
    net = tf.feature_column.input_layer(features, params['my_features'])
    for unit in params['hidden']:
        net = tf.layers.dense(net, unit)
    output_layer = tf.layers.dense(net, 1)
    
    # Bringing output to same dimension
    predictions = tf.squeeze(output_layer,1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions={"temp": predictions})
    
    average_loss = tf.losses.mean_squared_error(labels,predictions)
    total_loss = tf.to_float(tf.shape(labels)[0]) * average_loss
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=total_loss, 
            eval_metric_ops={'rmse':tf.metrics.root_mean_squared_error(labels,predictions)})
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = params.get("optimizer", tf.train.AdamOptimizer)
        optimizer = optimizer(params.get("learning_rate", None))
        train_op = optimizer.minimize(loss=average_loss, global_step=tf.train.get_global_step())

        return tf.estimator.EstimatorSpec(mode=mode, loss=total_loss, train_op=train_op)

In [71]:
def train_fn(features, labels, batch_size):
    
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    dataset = dataset.shuffle(2000).repeat().batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

def eval_fn(features, labels, batch_size):
    
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    dataset = dataset.batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [83]:
def load_data(path,columns):
    data = pd.read_csv(path)[-2000:].reset_index(drop=True)
    data = data[columns].astype('float32')
    data = pd.concat([data,data.shift(-1)],axis=1)[1:]
    return data.values, data.columns[:-1]

In [95]:
features,labels = load_data("Jena_time_series_data.csv",['T (degC)'])

In [96]:
my_features = [tf.feature_column.numeric_column(features)]

In [103]:
regressor.train(input_fn = lambda:train_fn(features,labels,32))

ValueError: Dimensions 1999 and 1 are not compatible